# Louisville Free Public Library

Analysis of Young Adult (YA) genre in the Louisville Free Public Library collection.

## Questions

In this analysis we will look at the following questions:

- How much was spent on the collection for YA? 
- How many books are in the collection for YA?
- How does YA spending compare to other collections?
- Did the spending on YA change over time?
- Is YA more or less popular at any of the locations?

### Load the clean library collection data and show a preview of the data

In [24]:
import pandas as pd
import numpy as np
from pathlib import Path

# load the clean books data into a dataframe and show the first few rows
books_data_path = Path('results/books_clean.csv.gz')
books_df = pd.read_csv(books_data_path)
books_df.head()

,BibNum,Title,Author,PublicationYear,ItemType,ItemCollection,ItemLocation,ItemPrice,Genre,Audience
0,707409,"Jeff Immelt and the new GE way : innovation, t...","Magee, David, 1965-",2009,Book,Adult Non-Fiction,Main,25.95,Non-Fiction,Adult
1,707411,Robin rescues dinner : 52 weeks of quick-fix m...,"Miller, Robin, 1964-",2009,Book,Adult Non-Fiction,Southwest,19.99,Non-Fiction,Adult
2,707411,Robin rescues dinner : 52 weeks of quick-fix m...,"Miller, Robin, 1964-",2009,Book,Adult Non-Fiction,Southwest,19.99,Non-Fiction,Adult
3,707411,Robin rescues dinner : 52 weeks of quick-fix m...,"Miller, Robin, 1964-",2009,Book,Adult Non-Fiction,Remote Shelving - Main,19.99,Non-Fiction,Adult
4,707411,Robin rescues dinner : 52 weeks of quick-fix m...,"Miller, Robin, 1964-",2009,Book,Adult Non-Fiction,Remote Shelving - Main,19.99,Non-Fiction,Adult


### How much was spent on the collection for YA?

In [1]:
# TODO: First figure out which records in the dataframe are YA using a mask
# YA = Genre: Fiction, Audience = Teen. Then slice the DataFrame using the mask 
# and sum the ItemPrice column and format the result.


The YA collection has a total cost of [TODO: insert answer here].

### How many books in  the collection are YA?

In [ ]:
# TODO: Create a new column in the dataframe called YA_Category, use 
# value_counts() to get the count and percent of YA books, and use the concat()
# function to combine the counts and percents into a single dataframe. 

YA accounts for [TODO: insert %YA here] percent of the total number of books in the collection.

### How does YA spending compare to other collections?

In [ ]:
# TODO: Group the data by Genre and Audience using groupby() and use sum() to 
# get the total cost. Format the totals as currency.

### Did the spending on YA change over time?

In [ ]:
# TODO: Calculate the counts (value_counts()), total cost (sum()), and average 
# cost (mean()) for all YA books by publication year using groupby(). 
# Concatenate the counts and costs using concat() into a single dataframe. 
# Format the counts and costs as numbers and currency.

Spending on YA books [TODO insert analysis here].

Is there a correlation between ItemPrice and PublicationYear?

In [ ]:
# TODO: Use the corr() function to determine if there is a correlation between
# the ItemPrice and PublicationYear.

[TODO: describe the correlation here.]

### Is YA more or less popular at any of the locations?

In [ ]:
# TODO: Calculate the total number of books by location and the number of YA
# books by location and concatenate them into a single DataFrame. Add a new column
# to show the % of books by location that are YA. Format the values appropriately.

[TODO: summarize the YA books by location here.]